In [3]:
#查询多单

In [1]:
# -*- coding:utf-8 -*-
import sys
sys.path.append('../../')
%matplotlib inline
stdout = sys.stdout
import JSONData.tdx_data_Day as tdd
import JSONData.tdx_hdf5_api as h5a
from JohnsonUtil import commonTips as cct

from ipykernel.iostream import OutStream
if isinstance(stdout,OutStream):
    sys.stdout = stdout
else:
    raise "no OutStream:%s"%(sys.stdout)
from  JSONData import sina_data 
import pandas as pd
# mpl.rcParams['font.sans-serif'] = ['STHeiti']
# mpl.rcParams['axes.unicode_minus'] = False
#中文字体显示  
import matplotlib.pyplot as plt
plt.ion()
plt.rc('font', family='SimHei', size=13)
from IPython.core.pylabtools import figsize
figsize(8, 5)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [2]:
def get_quotes_tdx(code, start=None, end=None, dl=120, resample='d', show_name=False,power=False):

        if resample in ['d', 'w', 'm']:
            quotes = tdd.get_tdx_append_now_df_api(code=code, start=start, end=end, dl=dl,power=power).sort_index(ascending=True)
        else:
            h5_fname = 'sina_MultiIndex_data'
            h5_table = 'all_10'
            time_s = time.time()
            h5 = h5a.load_hdf_db(h5_fname, table=h5_table, code_l=None, timelimit=False, dratio_limit=0.12)
            quotes = cct.get_limit_multiIndex_freq(h5, freq=resample.upper(), col='all', start=start, end=end, code=code)
            quotes = quotes.reset_index().set_index('ticktime')
            # period_stock_data['close'] = stock_data['close'].resample(period_type, how='last')
            if 'volume' in quotes.columns:
#                 quotes.rename(columns={'volume': 'vol'}, inplace=True)
                quotes['amount'] = (map(lambda x, y: round((x * y), 1), quotes.close.values, quotes.vol.values))            
        
        if not resample == 'd' and resample in tdd.resample_dtype:
            quotes = tdd.get_tdx_stock_period_to_type(quotes, period_day=resample)
        if str(quotes.index.dtype) <> 'datetime64[ns]':
            quotes.index = quotes.index.astype('datetime64')

        if show_name:
            if 'name' in quotes.columns:
                cname = quotes.name[0]
                # cname_g =cname
            else:
                dm = tdd.get_sina_data_df(code)
                if 'name' in dm.columns:
                    cname = dm.name[0]
                else:
                    cname = '-'
        else:
            cname = None
        if quotes is not None and len(quotes) > 0:
            quotes= quotes.loc[:, ['open', 'close', 'high', 'low', 'volume', 'amount']]
        else:
            # log.error("quotes is None check:%s"%(code))
            raise Exception("Code:%s error, df is None" % (code))
        return quotes

In [3]:
# import pandas as pd
# tpp='/Volumes/RamDisk/sina_MultiIndex_data.h5'
# spp=pd.HDFStore(tpp)
# code='002673'
# code='600559'
# code='002342'
code='300649'

# code='002691'
# code='000555'

resample='d'
# dfm = spp.all_10
# df_freq = cct.get_limit_multiIndex_freq(dfm)

h5_fname = 'sina_MultiIndex_data'
h5_table = 'all_10'
h5 = h5a.load_hdf_db(h5_fname, table=h5_table, code_l=None, timelimit=False, dratio_limit=0.12)
# h5 = h5.drop_duplicates('volume')
# h5.volume = h5.volume - h5.volume.shift(1)
print h5.loc[code][:2]
print h5.loc[code][-2:]
print h5.loc[code].lastbuy.value_counts()
print "median:",h5.loc[code].close.median()
df_freq = cct.get_limit_multiIndex_freq(h5,freq='5T', col='all',start='09:25:00',end=None, code=code)
df_tdx = get_quotes_tdx(code,dl=30,resample=resample)
print "last TDD:",df_tdx[-1:].amount/df_tdx[-1:].close,(df_tdx[-1:].high+df_tdx[-1:].low)/2,df_tdx[-1:].close
# #no use df_freq['volume'] = df_freq['volume'] - df_freq['volume'].shift(1)
# print df_freq.sort_values('volume',ascending=False)
print df_freq[:5]
print df_freq[-5:]

# print df_freq.low.cummin(),df_freq.volume.sum()/100
print "volume Max:",df_freq.sort_values('volume',ascending=False)[:3]
print "volume Min:",df_freq.sort_values('volume',ascending=False)[-3:]
# print df_freq.loc[:,['close','volume']][-30:]
df_freq['amount'] = (map(lambda x, y: round((x * y), 1), df_freq.close.values, df_freq.volume.values)) 
print "mean:%s mean2:%s median:%s"%(df_freq.amount.sum()/df_freq.volume.sum(),df_freq.close.mean(),df_freq.close.median())
# print df_freq.volume.value_counts().T
# df = tdd.get_tdx_Exp_day_to_df('999999',start=20170602,end=None).sort_index(ascending=True)
# print df[-2:]

                     close   high    low  llastp  volume  lastbuy
ticktime                                                         
2019-04-08 09:25:03  27.77  27.77  27.77   27.59    8400    27.77
2019-04-08 09:33:48  27.54  27.77  27.50   27.59  144400    27.77
                     close   high   low  llastp   volume  lastbuy
ticktime                                                         
2019-04-08 13:48:54  26.62  27.77  26.5   27.59  2205428    26.67
2019-04-08 13:50:03  26.64  27.77  26.5   27.59  2209628    26.67
27.77    42
27.01    34
27.22    33
27.30    31
27.56    22
26.67    20
27.05     1
Name: lastbuy, dtype: int64
median: 27.19
last TDD: date
2019-04-04    3912570.253715
dtype: float64 date
2019-04-04    27.26
dtype: float64 date
2019-04-04    27.59
Name: close, dtype: float64
                             high  close   open    low  volume
code   ticktime                                               
300649 2019-04-08 09:30:00  27.77  27.77  27.77  27.77    8400
     